In [2]:
import sqlite3

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import re
from fireworks.client import Fireworks
import logging

In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
fireworks_api_key = 'fw_3ZZL7o5gW7uXcibaK3woWnP6'
client = Fireworks(api_key=fireworks_api_key)

In [5]:
con = sqlite3.connect('careerbuilder.db')
cursor = con.cursor()

In [7]:
def clean_job_description(description: str) -> str:
    '''
    Clean the job descriptions up by removing all HTML elements and unwanted characters.
    :param description: raw description string
    :return: cleaned job description string
    '''
    if not isinstance(description, str) or description is None:
        return description

    try:
        soup = BeautifulSoup(description, 'html.parser')
        cleaned_desc = soup.get_text().strip()  # Extract text and remove leading/trailing spaces

        # Normalize Unicode text
        cleaned_desc = unicodedata.normalize('NFKC', cleaned_desc)

        # Replace problematic fraction symbols
        cleaned_desc = cleaned_desc.replace("â", "/")  # Fix 1⁄2 fraction separator
        cleaned_desc = cleaned_desc.replace("1⁄2", "1/2")  # Replace any remaining fraction symbols

        # Remove bullet points (• and other similar symbols)
        cleaned_desc = re.sub(r"[•●▪▶◾■◆◇]", "", cleaned_desc)

        # Encode and Decode to remove escaped characters
        cleaned_desc = cleaned_desc.encode().decode('unicode-escape')

        # Replace unwanted characters
        cleaned_desc = cleaned_desc.replace("\r", "").replace("\n", " ")  # Removes \r and \n

        # Remove extra spaces
        cleaned_desc = re.sub(r"\s+", " ", cleaned_desc).strip()

        return cleaned_desc
    except Exception as e:
        logging.error(f'Error while cleaning description: {e} \n returning original string')
        # Normalize unicode text
        cleaned_desc = unicodedata.normalize('NFKC', description)

        # Remove bullet points (• and other similar symbols)
        cleaned_desc = re.sub(r"[•●▪▶◾■◆◇]", "", cleaned_desc)

        # encode and decode to remove escaped characters
        cleaned_desc = cleaned_desc.encode().decode('unicode-escape')

        # Replace newline characters with spaces.
        cleaned_desc = cleaned_desc.replace("\n", " ")
        cleaned_desc = cleaned_desc.replace("\r", "")
        return cleaned_desc


In [8]:
user_1 = pd.read_sql('select * from users LIMIT 1', con = con)
user_1

,user_id,window_id,split,city,state,country,zipcode,degree_type,major,graduation_date,work_history_count,total_years_experience,currently_employed,managed_others,managed_how_many,generated_resume
0,47,1,Train,Paramount,CA,US,90723,High School,None,1999-06-01 00:00:00,3,10.0,Yes,No,0,SUMMARY:\nThe client is a highly motivated ind...


In [9]:
user_1_apps = pd.read_sql(f'SELECT * FROM apps WHERE user_id = {user_1["user_id"].item()}', con = con)
user_1_apps

,app_id,user_id,window_id,split,application_date,job_id
0,0,47,1,Train,2012-04-04 15:56:23.537,169528
1,1,47,1,Train,2012-04-06 01:03:00.003,284009
2,2,47,1,Train,2012-04-05 02:40:27.753,2121
3,3,47,1,Train,2012-04-05 02:37:02.673,848187
4,4,47,1,Train,2012-04-05 22:44:06.653,733748
5,5,47,1,Train,2012-04-05 02:34:40.223,576958
6,6,47,1,Train,2012-04-05 22:55:03.583,262470
7,7,47,1,Train,2012-04-05 02:38:49.52,602298


In [10]:
job_ids = [str(id) for id in user_1_apps["job_id"].to_list()]
query = f"SELECT * FROM jobs WHERE job_id IN ({', '.join(job_ids)})"
applied_jobs = pd.read_sql_query(query, con = con)

In [11]:
applied_jobs

,job_id,window_id,title,description,requirements,city,state,country,zip5,start_date,end_date
0,2121,1,MEDICAL- FRONT OFFICE,"Medical - Front Office\r\nIndustrial Clinic, ...",Please refer to the Job Description to view th...,Los Angeles,CA,US,90066,2012-03-09 18:07:59.063,2012-04-08 23:59:59
1,169528,1,"Resort Host/Marketing Coordinator - Anaheim, CA","<P STYLE=""MARGIN-TOP: 0px; MARGIN-BOTTOM: 0px""...",• Bachelor's Degree preferred;<BR> \r\n• Posse...,Anaheim,CA,US,92801,2012-04-03 22:31:26.41,2012-05-02 23:59:59
2,262470,1,Account Manager- Customer Service,Staffmark has partnered with an aerospace manu...,"<strong><span style=""text-decoration: underlin...",Buena Park,CA,US,90620,2012-04-04 17:07:51.35,2012-05-03 23:59:59
3,284009,1,Administrative Assistant,<p><strong>Administrative Assistant $60K - $65...,Requirement:&nbsp; 2-4 years of significant ex...,Los Angeles,CA,US,90036,2012-03-26 00:22:39.093,2012-04-25 23:59:59
4,576958,1,RECEPTIONIST,"WE NEED A SPIRITED, OUT GOING PERSON TO JOIN O...",GREET ALL PATIENTS WITH A SMILE AND BY NAME <...,San Gabriel,CA,US,91775,2012-03-11 11:43:48.003,2012-04-10 23:59:59
5,602298,1,Customer Service Rep.,Customer Service Customer Service Rep. SANTA M...,Please refer to the Job Description to view th...,Rancho Dominguez,CA,US,90220,2012-03-23 16:10:51.293,2012-04-22 23:59:59
6,733748,1,Administrative Assistant,One of the largest oilfield services companies...,None,Long Beach,CA,US,None,2012-03-07 06:43:12.37,2012-04-06 23:59:59
7,848187,1,Administrative Assistant,<DIV>\r<P><SPAN>ADMINISTRATIVE ASSISTANT</SPAN...,"<P><SPAN style=""TEXT-DECORATION: underline""></...",Los Angeles,CA,US,90071,2012-03-19 17:55:56.67,2012-04-18 23:59:59


In [12]:
applied_jobs['cleaned_description'] = applied_jobs.apply(lambda x: clean_job_description(x['description']), axis=1)
applied_jobs['cleaned_requirements'] = applied_jobs.apply(lambda x: clean_job_description(x['requirements']), axis=1)
applied_jobs = applied_jobs.replace(r'^\s*$', None, regex=True)
applied_jobs

,job_id,window_id,title,description,requirements,city,state,country,zip5,start_date,end_date,cleaned_description,cleaned_requirements
0,2121,1,MEDICAL- FRONT OFFICE,"Medical - Front Office\r\nIndustrial Clinic, ...",Please refer to the Job Description to view th...,Los Angeles,CA,US,90066,2012-03-09 18:07:59.063,2012-04-08 23:59:59,"Medical - Front Office Industrial Clinic, Full...",Please refer to the Job Description to view th...
1,169528,1,"Resort Host/Marketing Coordinator - Anaheim, CA","<P STYLE=""MARGIN-TOP: 0px; MARGIN-BOTTOM: 0px""...",• Bachelor's Degree preferred;<BR> \r\n• Posse...,Anaheim,CA,US,92801,2012-04-03 22:31:26.41,2012-05-02 23:59:59,Wyndham Vacation Ownership is the world's larg...,Bachelor's Degree preferred; Possess excellent...
2,262470,1,Account Manager- Customer Service,Staffmark has partnered with an aerospace manu...,"<strong><span style=""text-decoration: underlin...",Buena Park,CA,US,90620,2012-04-04 17:07:51.35,2012-05-03 23:59:59,Staffmark has partnered with an aerospace manu...,Experience with distribution or manufacturing ...
3,284009,1,Administrative Assistant,<p><strong>Administrative Assistant $60K - $65...,Requirement:&nbsp; 2-4 years of significant ex...,Los Angeles,CA,US,90036,2012-03-26 00:22:39.093,2012-04-25 23:59:59,Administrative Assistant $60K - $65KOur client...,Requirement: 2-4 years of significant experien...
4,576958,1,RECEPTIONIST,"WE NEED A SPIRITED, OUT GOING PERSON TO JOIN O...",GREET ALL PATIENTS WITH A SMILE AND BY NAME <...,San Gabriel,CA,US,91775,2012-03-11 11:43:48.003,2012-04-10 23:59:59,"WE NEED A SPIRITED, OUT GOING PERSON TO JOIN O...",GREET ALL PATIENTS WITH A SMILE AND BY NAME SC...
5,602298,1,Customer Service Rep.,Customer Service Customer Service Rep. SANTA M...,Please refer to the Job Description to view th...,Rancho Dominguez,CA,US,90220,2012-03-23 16:10:51.293,2012-04-22 23:59:59,Customer Service Customer Service Rep. SANTA M...,Please refer to the Job Description to view th...
6,733748,1,Administrative Assistant,One of the largest oilfield services companies...,None,Long Beach,CA,US,None,2012-03-07 06:43:12.37,2012-04-06 23:59:59,One of the largest oilfield services companies...,None
7,848187,1,Administrative Assistant,<DIV>\r<P><SPAN>ADMINISTRATIVE ASSISTANT</SPAN...,"<P><SPAN style=""TEXT-DECORATION: underline""></...",Los Angeles,CA,US,90071,2012-03-19 17:55:56.67,2012-04-18 23:59:59,"ADMINISTRATIVE ASSISTANT A&E Support Services,...",None


In [13]:
def build_LLM_query(query_dicts: dict) -> str:
    '''
    Build the query for the LLM using the instruction prompt and all the details from the user and their interactions.
    :param query_dicts: Dict where each key is a user_id and each value is a Dict of query components.
    :return: Dict[str] where the keys are the user_id's and the items are query strings
    '''

    if not isinstance(query_dicts, dict):
        raise TypeError("query_dicts is not a dictionary!")

    for key, value in query_dicts.items():
        if not isinstance(value, dict):
            raise TypeError(f"query_dicts[{key}] is not a dictionary but {type(value)}")

    final_queries = {}
    for key, item in query_dicts.items():
        query_components = [
            "You are a career advisor tasked with writing a resume for a client given their categorical descriptors, their job history, and the types of jobs they have applied at recently (and therefore find desirable).\nThe client’s description begins and ends with [CLIENT] [/CLIENT] tokens, respectively.\n\n[CLIENT]\nCategorical Descriptors:"
        ]

        # Add categorical descriptors from user table
        if item['degree_type'] is not None:
            query_components.append(f'Highest Education: {item["degree_type"]}')
        if item['major'] is not None:
            query_components.append(f'Education Major: {item["major"]}')
        if item['graduation_date'] is not None:
            query_components.append(f'Graduation Date: {item["graduation_date"]}')
        if item['total_years_experience'] is not None:
            query_components.append(f'Total Years of Experience: {item["total_years_experience"]}')
        if item['currently_employed'] is not None:
            query_components.append(f'Currently Employed: {item["currently_employed"]}')
        if item['managed_others'] is not None:
            query_components.append(f'Has Experience Managing Others: {item["managed_others"]}')
        if item['managed_how_many'] is not None:
            query_components.append(f'Number of People Managed: {item["managed_how_many"]}')

        # Add job titles from user_history table
        query_components.append('Job History (from most recent to furthest ago):')

        if item['job_history'] is not None:
            sorted_jobs = sorted(item['job_history'], key = lambda x: int(x['sequence']))
            for job in sorted_jobs:
                if job['job_title'] is not None:
                    query_components.append(job["job_title"])
        else:
            query_components.append('No job history.')

        # Add job descriptions of jobs the user has applied to
        query_components.append('Job descriptions of jobs the user has recently applied to (most recent to furthest ago):')

        if item['job_applications'] is not None:
            for job in item['job_applications']:
                if job['title'] is not None:
                    query_components.append(f'Title: {job["title"]}')
                if job['description'] is not None:
                    query_components.append(f'Description: {job["description"]}')
                if job['requirements'] is not None:
                    query_components.append(f'Requirements: {job["requirements"]}')
        else:
            query_components.append('No job applications.')

        query_components.append("[/CLIENT]\n")
        query_components.append("Answer the following Questions:\n1. What are some likely tasks the client accomplished in their previous job roles? Use their titles to infer tasks.\n2. What skills does the user likely have from their previous job roles and education?\n3. Tailor the client’s skills to the jobs they are applying at. Assume the client has some skills similar to those required at the jobs they are applying to.\n4. What is a good summary of the client’s previous job roles, skills, and jobs they are applying to?\n")
        query_components.append("Fill in the following resume template with the answers to those questions:\n\n[BEGIN]\nSUMMARY:\n(Summary of the user, their desires, and their skills)\n\nWORK EXPERIENCE:\n(List job title and tasks they likely accomplished at the jobs)\n\nEDUCATION:\n(List their education)\n\nSKILLS:\n(Infer skills from their work experience, education, and managerial responsibilities)\n[END]\n\nOnly reply with that format, no boilerplate language. Limit the output to 400 tokens.")

        full_query = '\n'.join(query_components)
        final_queries[key] = full_query

    return final_queries

In [14]:
def orchestrate_LLM_query_build_batch(user_ids: list):
    try:
        # Convert list to a comma-separated string for SQL
        user_ids_str = ', '.join(map(str, user_ids))

        # Fetch all users at once
        user_records = pd.read_sql_query(f'SELECT * FROM users WHERE user_id IN ({user_ids_str})', con=con)

        # Fetch all applications at once
        applications_records = pd.read_sql_query(f'SELECT * FROM apps WHERE user_id IN ({user_ids_str})', con=con)

        # Fetch all job history at once
        user_history = pd.read_sql_query(f'SELECT * FROM user_history WHERE user_id IN ({user_ids_str})', con=con)

        # Fetch all job postings applied to (batch query instead of one per user)
        applied_jobs_query = f"WITH cte AS (SELECT apps.user_id AS user_id, apps.split AS split, DATETIME(apps.application_date) AS application_date, apps.job_id AS job_id, jobs.title AS title, jobs.description AS description, jobs.requirements AS requirements, ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY application_date DESC) AS date_rank FROM apps INNER JOIN jobs ON apps.job_id = jobs.job_id WHERE apps.user_id IN ({user_ids_str})) SELECT * FROM cte WHERE date_rank <= 10;"
        applied_jobs = pd.read_sql_query(applied_jobs_query, con=con)

        # Clean descriptions in bulk
        applied_jobs['cleaned_description'] = applied_jobs['description'].apply(clean_job_description)
        applied_jobs['cleaned_requirements'] = applied_jobs['requirements'].apply(clean_job_description)

        # Replace empty strings with None
        applied_jobs = applied_jobs.replace(r'^\s*$', None, regex=True)

        # Build query dictionaries for all users
        query_dicts = {}
        for user_id in user_ids:
            user_data = user_records[user_records['user_id'] == user_id]
            user_apps = applications_records[applications_records['user_id'] == user_id]
            user_hist = user_history[user_history['user_id'] == user_id]
            user_jobs = applied_jobs[applied_jobs['job_id'].isin(user_apps['job_id'])]

            query_dicts[user_id] = {
                'degree_type': user_data['degree_type'].iloc[0],
                'major': user_data['major'].iloc[0],
                'graduation_date': pd.to_datetime(user_data['graduation_date']).dt.strftime('%b %-d, %Y').iloc[0],
                'total_years_experience': str(user_data['total_years_experience'].iloc[0]),
                'currently_employed': user_data['currently_employed'].iloc[0],
                'managed_others': user_data['managed_others'].iloc[0],
                'managed_how_many': str(user_data['managed_how_many'].iloc[0]),
                'job_history': user_hist[['sequence', 'job_title']].astype(str).to_dict(orient='records'),
                'job_applications': user_jobs[['title', 'cleaned_description', 'cleaned_requirements']]
                .rename(columns={'cleaned_description': 'description', 'cleaned_requirements': 'requirements'})
                .to_dict(orient='records')
            }

        return query_dicts

    except Exception as e:
        logging.error(f'Error in orchestrating LLM query: {e}')

In [17]:
def fireworks_api_call(query: str, user_id: int, model_name: str = 'accounts/fireworks/models/llama-v3p3-70b-instruct'):
    """
    Make calls to Fireworks API and prompt the LLM for a response. The paramaters are tunable and the current set is based on experimentation.
    :param query: Query string to pass to the LLM.
    :param user_id: User ID taken for logging purposes.
    :param model_name: model path to use.
    :return: A string containing the LLM-generated resume.
    """
    try:
        response = client.completion.create(
            model=model_name,
            stream=False,
            prompt=query,
            max_tokens=627,
            temperature=round(np.random.uniform(1.3, 1.8), 1),
            top_p=1,
            top_k=40,
            presence_penalty=0,
            frequency_penalty=0,
            echo=False
        )
        return response.choices[0].text

    except Exception as e:
        logging.error(f'Error in Fireworks API call for user {user_id}: {e}')

The following section runs the loop to query the LLM API for resumes and store them in the SQLite database. The queries are batched for efficiency and robustness.

In [15]:
# Create sample of unique users so I'm not running 350k queries through the API
# Also, filter users list so that I'm not taking users with abnormal amounts of applications (right-tailed distribution)
# Added a filter to restrict to window 1
users_for_gen_query = "WITH cte AS (SELECT user_id, COUNT(DISTINCT(job_id)) AS n_apps FROM apps GROUP BY user_id HAVING n_apps <= 100) SELECT u.user_id, u.split, c.n_apps FROM users u INNER JOIN cte c ON u.user_id = C.user_id WHERE u.split = 'Train' AND u.window_id = 1;"
unique_users = pd.read_sql_query(users_for_gen_query, con = con)
sample_of_unique_users = unique_users.sample(n=2000).reset_index(drop=True)['user_id'].to_list()

In [18]:
batch_size = 50

for i in range(0, len(sample_of_unique_users), batch_size):
    response_list = []

    # Pull the next batch of user_ids
    batch = sample_of_unique_users[i:i + batch_size]

    # Fetch data for the batch
    query_dicts = orchestrate_LLM_query_build_batch(batch)
    queries = build_LLM_query(query_dicts)

    # Generate resumes for the batch
    for user_id, query in queries.items():     # This should probably be passing in the entire dict[dict], not just dict.
        try:
            response = fireworks_api_call(query=query, user_id=user_id)
            update_text = response.split('[BEGIN]')[1].split('[END]')[0].strip()
            response_list.append((update_text, user_id))
        except Exception as e:
            logging.error(f'Error in Fireworks API call for user {user_id}: {e}')
            continue

    # Update SQLite database in batches
    cursor.executemany('UPDATE users SET generated_resume=? WHERE user_id=?', response_list)
    con.commit()

    logging.info(f'Successfully generated and stores resumes for batch {(i // batch_size)+1}/{(len(sample_of_unique_users)//batch_size)}.')

2025-02-27 11:14:52,822 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"
2025-02-27 11:14:58,454 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"
2025-02-27 11:15:06,238 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"
2025-02-27 11:15:16,476 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"
2025-02-27 11:15:21,085 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"
2025-02-27 11:15:26,001 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"
2025-02-27 11:15:29,994 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"
2025-02-27 11:15:33,372 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"
2025-02-27 11:15:36,753 - INFO - HTTP Re

In [49]:
test_query_dict = orchestrate_LLM_query_build_batch([72])
test_query = build_LLM_query(test_query_dict)
# test_query
test_response = fireworks_api_call(query=test_query[72], user_id=72)

try:
    update_db = test_response.split('[BEGIN]')[1].split('[END]')[0].strip()
    cursor.execute('UPDATE users SET generated_resume=? WHERE user_id=72', (update_db,))
    con.commit()
    cursor.execute('SELECT * FROM users WHERE user_id=72').fetchall()
except Exception as e:
    print(test_response)
    logging.error(f'Error in orchestrating LLM query for user: {e}')

2025-02-24 14:52:43,612 - INFO - HTTP Request: POST https://api.fireworks.ai/inference/v1/completions "HTTP/1.1 200 OK"


In the following section, I am trying to see what the median number of applications is per user WHO HAS a generated resume. This will inform the cutoff score for classifying users as highly active appliers (1) or not highly active appliers (0).

In [33]:
median_apps_query = '''
SELECT
    user_id,
    COUNT(*) AS n_apps
FROM
    apps
WHERE
    user_id IN (
        SELECT
            user_id
        FROM
            users
        WHERE
            generated_resume NOT NULL
        )
GROUP BY user_id;'''


apps_stats = pd.read_sql_query(median_apps_query, con=con)

In [34]:
apps_stats.describe()

,user_id,n_apps
count,5.004000e+03,5004.000000
mean,7.405433e+05,4.440248
std,4.220730e+05,7.250702
min,4.700000e+01,1.000000
25%,3.701500e+05,1.000000
50%,7.460575e+05,2.000000
75%,1.106938e+06,5.000000
max,1.472090e+06,100.000000


The median number of applications for this sample is 2. I think I will make the cutoff a bit higher at 5, which is the 75% quartile.

In [39]:
positive_class_users = apps_stats.loc[apps_stats['n_apps'] >= 5]['user_id'].to_list()
negative_class_users = apps_stats.loc[apps_stats['n_apps'] < 5]['user_id'].to_list()

positive_class_df = pd.read_sql_query(f'SELECT user_id, generated_resume FROM users WHERE user_id IN ({", ".join(map(str, positive_class_users))})', con=con)
negative_class_df = pd.read_sql_query(f'SELECT user_id, generated_resume FROM users WHERE user_id IN ({", ".join(map(str, negative_class_users))})', con=con)

At this point, I'm going to export the user dataframes to parquet so that I can upload them to Google Colab and make use of the GPU instances there.

In [44]:
positive_class_df.to_parquet('./positive_class.parquet')
negative_class_df.to_parquet('./negative_class.parquet')